## Hadoop Combiner

   Используется для уменьшениы выходных данных маппера, тем самым уменьшает сетевую нагрузки между мапперами и редьюсерами. В основном применяется для неравномерно распределенных данных, и тех данных, у которых количество значений соотвествующих определенному ключу очень много, проядка 100000 или больше. И редьюсер реализует дистрибутивную функцию, например, вычисление максимума, минимума или суммирование значений. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Hadoop Combiner")
hdfs = "hdfs://localhost:9000/"

#### Пример № 1. Реализация MapReduce-задачи в интерфейсе Hadoop Streaming. Вычисление средного значения количества пунктов в формуле патента  патентообладателей определенной страны.
Это пример неравномерного распределения данных, так как больше всего патентов за период 1960 - 2000 было поданно США. В маппере возмем поле стран за ключи, значения за 1. 

In [3]:
# average_by_attribute_mapper.py

import sys

contry_index = 4 # CONTRY - страна патентаобладателя
claims_index = 8 # CLAIMS - количество пунктов в формуле патента

for line in sys.stdin:
    fields = line.split(",")
    if fields[claims_index] and fields[claims_index].isdigit():
        print(fields[contry_index][1:-1] + '\t' + fields[claims_index])


In [ ]:
hadoop jar $HADOOP_HOME/libexec/share/hadoop/tools/lib/hadoop-streaming-2.8.0.jar \ 
    -D mapreduce.job.reduces=1 \ 
    -input /user/askar/apat63_99.txt \
    -output /user/askar/output \ 
    -mapper "python3 /Users/shabykov/Hadoop/average_by_attribute_mapper.py" \
    -file /Users/shabykov/Hadoop/average_by_attribute_mapper.py

In [7]:
out_data = sc.textFile(hdfs + "/user/askar/output")

In [8]:
for val in out_data.collect()[:32]:
    print(val)

AD	7
AD	14
AD	28
AD	12
AD	9
AE	4
AE	12
AE	24
AE	16
AE	11
AE	35
AE	16
AE	20
AE	10
AE	7
AE	23
AE	26
AE	11
AE	12
AE	4
AG	20
AG	7
AG	8
AG	12
AG	3
AG	24
AG	14
AG	18
AI	10
AM	18
AN	3
AN	26


Reducer считает среднее значение данных по опредделенному ключу
Combiner = Reducer (в качестве комбайнера будем использовать редьюсер)

In [9]:
# average_by_attribute_reducer.py

import sys

last_key, count, summ = None, 0, 0.0
for line in sys.stdin:
    (key, val) = line.split("\t")
    
    if last_key and last_key != key:
        print("{}\t{}".format(last_key, summ / count))
        summ, count = 0.0, 0
    
    last_key = key
    summ += float(val)
    count += 1

if last_key:
    print("{}\t{}".format(last_key, summ / count))

In [ ]:
hadoop jar $HADOOP_HOME/libexec/share/hadoop/tools/lib/hadoop-streaming-2.8.0.jar \ 
    -D mapreduce.job.reduces=1 \ 
    -input /user/askar/apat63_99.txt \
    -output /user/askar/output \ 
    -mapper "python3 /Users/shabykov/Hadoop/average_by_attribute_mapper.py" \
    -combiner "python3 /Users/shabykov/Hadoop/average_by_attribute_reducer.py" \
    -file /Users/shabykov/Hadoop/average_by_attribute_mapper.py

In [10]:
out_data = sc.textFile(hdfs + "/user/askar/output")

Результаты на выходе комбайнеров: 

In [12]:
for val in out_data.collect()[:50]:
    print(val)

AD	7.0
AD	15.75
AE	15.5
AE	15.2
AG	13.5
AG	13.166666666666666
AI	10.0
AM	18.0
AN	11.333333333333334
AN	4.5
AR	8.191489361702128
AR	10.258555133079849
AT	12.070684314277669
AT	9.46241627387745
AU	10.206333973128599
AU	14.1228402865571
AW	15.5
AZ	11.0
BB	13.0
BB	10.666666666666666
BE	11.14691943127962
BE	12.64612374663732
BG	5.68
BG	4.829721362229102
BH	5.0
BH	8.0
BM	10.8
BM	9.625
BN	9.0
BO	8.777777777777779
BO	20.666666666666668
BR	10.23339317773788
BR	7.894894894894895
BS	13.783333333333333
BS	18.5
BY	15.0
BZ	28.0
CA	10.516544655929723
CA	13.705782951558392
CC	9.0
CD	14.0
CD	6.0
CH	10.744842439356155
CH	12.919727088948788
CI	7.666666666666667
CK	8.0
CL	15.582089552238806
CL	11.4
CN	13.222222222222221
CN	10.862003780718336


In [ ]:
hadoop jar $HADOOP_HOME/libexec/share/hadoop/tools/lib/hadoop-streaming-2.8.0.jar \ 
    -D mapreduce.job.reduces=1 \ 
    -input /user/askar/apat63_99.txt \
    -output /user/askar/output \ 
    -mapper "python3 /Users/shabykov/Hadoop/average_by_attribute_mapper.py" \
    -combiner "python3 /Users/shabykov/Hadoop/average_by_attribute_reducer.py"\
    -reducer "python3 /Users/shabykov/Hadoop/average_by_attribute_reducer.py"\ 
    -file /Users/shabykov/Hadoop/average_by_attribute_mapper.py

Результаты на выходе редьюсеров после использования комбайнеров:

In [13]:
out_data = sc.textFile(hdfs + "/user/askar/output")

In [14]:
for val in out_data.collect()[:50]:
    print(val)

AD	11.375
AE	15.35
AG	13.333333333333332
AI	10.0
AM	18.0
AN	7.916666666666667
AR	9.225022247390989
AT	10.76655029407756
AU	12.16458712984285
AW	15.5
AZ	11.0
BB	11.833333333333332
BE	11.896521588958471
BG	5.25486068111455
BH	6.5
BM	10.2125
BN	9.0
BO	14.722222222222223
BR	9.064144036316389
BS	16.141666666666666
BY	15.0
BZ	28.0
CA	12.111163803744057
CC	9.0
CD	10.0
CH	11.832284764152472
CI	7.666666666666667
CK	8.0
CL	13.491044776119402
CN	12.042113001470279
CO	12.044117647058822
CR	11.616666666666667
CS	7.60668016194332
CU	9.37784090909091
CY	10.785714285714286
CZ	12.823529411764707
DE	11.131700257745276
DK	10.832334668499442
DO	10.642857142857142
DZ	14.0
EC	12.13888888888889
EE	17.0
EG	11.73970588235294
ES	8.6256708539627
FI	10.248721177385207
FO	8.0
FR	10.97657827839095
GB	11.503545243661414
GE	7.5
GF	10.0


К сожалению, ответ не соотвествует реальности, так как редьюсер реализует недистрибутивную функцию. Необходимо поменять логику работы мапперов и написать отдельный комбайнер. В новый распеределитель на выходе будет выдавать: 

    - key - страна
    - value - значение количества пунктов, количество таких записей.  

<h5>Mapper:</h5>

In [4]:
# average_by_attribute_mapper.py

import sys

contry_index = 4 # CONTRY - страна патентаобладателя
claims_index = 8 # CLAIMS - количество пунктов в формуле патента

for line in sys.stdin:
    fields = line.split(",")
    if fields[claims_index] and fields[claims_index].isdigit():
        print("{}\t{},{}".format(fields[contry_index][1:-1], fields[claims_index], 1))

<h5>Combiner:</h5>

In [5]:
# average_by_attribute_combiner.py

import sys


last_key, count, summ = None, 0, 0.0
for line in sys.stdin:
    (key, value) = line.split("\t")
    (s, c) = value.split(",")
    
    if last_key and last_key != key:
        print("{}\t{},{}".format(last_key, summ, count))
        summ, count = 0.0, 0
    
    last_key = key
    summ += float(s)
    count += int(c)

if last_key:
    print("{}\t{},{}".format(last_key, summ, count))



<h5>Reducer:</5>

In [6]:
# average_by_attribute_reducer.py

import sys

last_key, count, summ = None, 0, 0.0

for line in sys.stdin:
    (key, value) = line.split("\t")
    (s, c) = value.split(",")
    
    if last_key and last_key != key:
        print("{}\t{}".format(last_key, summ / count))
        summ, count = 0.0, 0
    
    last_key = key
    summ += float(s)
    count += int(c)

if last_key:
    print("{}\t{}".format(last_key, summ / count))

In [ ]:
hadoop jar $HADOOP_HOME/libexec/share/hadoop/tools/lib/hadoop-streaming-2.8.0.jar \ 
    -D mapreduce.job.reduces=1 \ 
    -input /user/askar/apat63_99.txt \
    -output /user/askar/output \ 
    -mapper "python3 /Users/shabykov/Hadoop/average_by_attribute_mapper.py" \
    -combiner "python3 /Users/shabykov/Hadoop/average_by_attribute_combiner.py"\
    -reducer "python3 /Users/shabykov/Hadoop/average_by_attribute_reducer.py"

In [7]:
out_data = sc.textFile(hdfs + "/user/askar/output")

In [8]:
for val in out_data.collect()[:50]:
    print(val)

AD	14.0
AE	15.4
AG	13.25
AI	10.0
AM	18.0
AN	9.625
AR	9.188990825688073
AT	10.683988393563704
AU	12.291563832174107
AW	15.5
AZ	11.0
BB	11.0
BE	11.945544554455445
BG	4.989949748743719
BH	6.5
BM	10.076923076923077
BN	9.0
BO	11.75
BR	9.358426966292134
BS	15.778846153846153
BY	15.0
BZ	28.0
CA	12.286875635112953
CC	9.0
CD	10.0
CH	11.619630031169535
CI	7.666666666666667
CK	8.0
CL	13.901785714285714
CN	11.23407643312102
CO	12.057142857142857
CR	11.454545454545455
CS	6.77487922705314
CU	10.222222222222221
CY	11.619047619047619
CZ	12.823529411764707
DE	11.051392049883086
DK	10.903726708074535
DO	10.642857142857142
DZ	14.0
EC	12.476190476190476
EE	17.0
EG	11.91891891891892
ES	8.733971997052322
FI	10.738573407202216
FO	8.0
FR	10.943602074030457
GB	11.322297099326105
GE	7.5
GF	10.0


Получили правильные результаты! 

ВАЖНО СЛЕДИТЬ ЗА ДАННЫМИ НА ВЫХОДЕ МАППЕРОВ И КОМБАЙНЕРОВ ОНИ ДОЛЖНЫ БЫТЬ ОДНОГО ТИПА ИЛИ ФОРМАТА! Так-как комбайнер может выполняться так и не выполняться, все зависить от обьема данных по определенному ключу. Это проще отследить в Java API.

#### Пример № 1. Реализация MapReduce-задачи в интерфейсе Java API. Вычисление средного значения количества пунктов в формуле патента  патентообладателей определенной страны.
Это пример неравномерного распределения данных, так как больше всего патентов за период 1960 - 2000 было поданно США. В маппере возмем поле стран за ключи, значения за 1. 

<h5>
Код реализации (AverageByAttribute.java): 
</h5>

In [ ]:
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.conf.Configured;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.DoubleWritable;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat;
import org.apache.hadoop.mapreduce.lib.output.TextOutputFormat;
import org.apache.hadoop.util.Tool;
import org.apache.hadoop.util.ToolRunner;

import java.io.IOException;

/**
 * @author Askar Shabykov
 * @since 01.06.17
 */

public class AverageByAttribute extends Configured implements Tool {


    // Mapper
    public static class AverageByAttributeMapper extends Mapper<LongWritable, Text, Text, Text> {

        public void map(LongWritable key, Text value, Context context) throws IOException, InterruptedException {

            String[] fields = value.toString().split(",", -20);
            String country = fields[4];
            String numClaims = fields[8];
            if (numClaims.length() > 0 && !numClaims.startsWith("\"")) {
                context.write(new Text(country), new Text(numClaims + ",1"));
            }
        }
    }


    // Combiner
    public static class AverageByAttributeCombiner extends Reducer<Text, Text, Text, Text> {
        public void reduce(Text key, Iterable<Text> values, Context context) throws IOException, InterruptedException {

            double sum = 0;
            int count = 0;
            for (Text value : values) {
                String[] fields = value.toString().split(",");
                sum += Double.parseDouble(fields[0]);
                count += Integer.parseInt(fields[1]);
            }
            context.write(key, new Text(String.valueOf(sum) + "," + String.valueOf(count)));
        }
    }


    // Reducer
    public static class AverageByAttributeReducer extends Reducer<Text, Text, Text, DoubleWritable> {
        public void reduce(Text key, Iterable<Text> values, Context context) throws IOException, InterruptedException {
            double sum = 0;
            int count = 0;
            for (Text value : values) {
                String[] fields = value.toString().split(",");
                sum += Double.parseDouble(fields[0]);
                count += Integer.parseInt(fields[1]);
            }
            context.write(key, new DoubleWritable(sum / count));
        }
    }


    public int run(String[] args) throws IOException, ClassNotFoundException, InterruptedException {
        Configuration configuration = getConf(); // конфигурации воркера

        Job job = Job.getInstance(configuration, "Average By Attribute"); /// создаем воркера
        job.setJarByClass(this.getClass());

        Path in_put = new Path(args[0]); // путь к входным файлам в hdfs
        Path out_put = new Path(args[1]); // путь к выходному файлу в hdfs

        TextInputFormat.addInputPath(job, in_put);
        TextOutputFormat.setOutputPath(job, out_put);

        job.setMapperClass(AverageByAttributeMapper.class); // mapper class
        job.setCombinerClass(AverageByAttributeCombiner.class); // combiner class
        job.setReducerClass(AverageByAttributeReducer.class); // reducer class

        job.setInputFormatClass(TextInputFormat.class);
        job.setOutputFormatClass(TextOutputFormat.class);

        job.setOutputKeyClass(Text.class); // тип ключа на выходе маппера
        job.setOutputValueClass(Text.class); // тип значения на выходе маппера

        return job.waitForCompletion(true) ? 0 : 1;
    }

    public static void main(String[] args) throws Exception {
        int exitCode = ToolRunner.run(new Configuration(), new AverageByAttribute(), args);
        System.exit(exitCode);
    }
}


<h5>
pom.xml
</h5>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>

    <groupId>AverageByAttribute</groupId>
    <artifactId>AverageByAttribute</artifactId>
    <version>1.0-SNAPSHOT</version>

    <dependencies>
        <dependency>
            <groupId>org.apache.hadoop</groupId>
            <artifactId>hadoop-client</artifactId>
            <version>2.8.0</version>
            <scope>provided</scope>
        </dependency>
    </dependencies>

</project>

In [ ]:
 hadoop jar AverageByAttribute-1.0-SNAPSHOT.jar \
    AverageByAttribute \
    /user/askar/apat63_99.txt \
    /user/askar/output

In [9]:
out_data = sc.textFile(hdfs + "/user/askar/output")

In [10]:
for val in out_data.collect()[:50]:
    print(val)

"AD"	14.0
"AE"	15.4
"AG"	13.25
"AI"	10.0
"AM"	18.0
"AN"	9.625
"AR"	9.188990825688073
"AT"	10.683988393563704
"AU"	12.291563832174107
"AW"	15.5
"AZ"	11.0
"BB"	11.0
"BE"	11.945544554455445
"BG"	4.989949748743719
"BH"	6.5
"BM"	10.076923076923077
"BN"	9.0
"BO"	11.75
"BR"	9.358426966292134
"BS"	15.778846153846153
"BY"	15.0
"BZ"	28.0
"CA"	12.286875635112953
"CC"	9.0
"CD"	10.0
"CH"	11.619630031169535
"CI"	7.666666666666667
"CK"	8.0
"CL"	13.901785714285714
"CN"	11.23407643312102
"CO"	12.057142857142857
"CR"	11.454545454545455
"CS"	6.77487922705314
"CU"	10.222222222222221
"CY"	11.619047619047619
"CZ"	12.823529411764707
"DE"	11.051392049883086
"DK"	10.903726708074535
"DO"	10.642857142857142
"DZ"	14.0
"EC"	12.476190476190476
"EE"	17.0
"EG"	11.91891891891892
"ES"	8.733971997052322
"FI"	10.738573407202216
"FO"	8.0
"FR"	10.943602074030457
"GB"	11.322297099326105
"GE"	7.5
"GF"	10.0


In [11]:
sc.stop()